## Fragment long viral RNA 

In [1]:
from rouskinhf import convert
import pandas as pd
import numpy as np

from src import compare_frags

### SARS

In [2]:
## Import data from Sun

# Import ct file
data = convert(
    'ct',
    'data/SARS2/Sun/',
    name='SARS2',
    path_out='',
    filter=False
)
df = pd.DataFrame(data).T
sequence_Sun = df['sequence']['SARS2'].upper().replace('T','U')
paired_bases_Sun = np.array(df['structure']['SARS2'])


# Import shape file
UKN = -1000
data_shape = pd.read_excel('data/SARS2/Sun/GSE153984_SARS2-icSHAPE.xlsx', header=None)
data_shape.fillna(UKN, inplace=True)

full_shape_Sun = data_shape[2].values
assert ''.join(data_shape[1].tolist()).upper().replace('T','U') == sequence_Sun, 'Sequence mismatch'

Parsing ct files:   0%|          | 0/1 [00:00<?, ?it/s]

Parsing ct files: 100%|██████████| 1/1 [00:00<00:00, 32.51it/s]


Drop 0 datapoints with None values (null sequence or reference)


In [3]:
## Import data from Lan

# Import ct file
data = convert(
    'ct',
    'data/SARS2/Lan/',
    name='SARS',
    path_out='',
    filter=False
)
df = pd.DataFrame(data).T
sequence_Lan = df['sequence']['SARS']
paired_bases_Lan = np.array(df['structure']['SARS'])

# Import dms file
UKN = -1000
data_dms = pd.read_excel('data/SARS2/Lan/SARS_DMS.xlsx')
data_dms.fillna(UKN, inplace=True)

full_dms_Lan = data_dms['Vero (filtered)'].values
max_dms = np.median(full_dms_Lan[full_dms_Lan!=UKN][full_dms_Lan[full_dms_Lan!=UKN]>np.percentile(full_dms_Lan[full_dms_Lan!=UKN], 95)])
full_dms_Lan[full_dms_Lan!=UKN] =  full_dms_Lan[full_dms_Lan!=UKN] / max_dms
full_dms_Lan[full_dms_Lan>1] = 1

assert ''.join(data_dms['Nucleotide'].tolist()) == sequence_Lan, 'Sequence mismatch'

Parsing ct files: 100%|██████████| 1/1 [00:00<00:00, 38.09it/s]


Drop 0 datapoints with None values (null sequence or reference)


In [5]:
gold_data = compare_frags([sequence_Lan, sequence_Sun], [full_dms_Lan, full_shape_Sun], [paired_bases_Lan, paired_bases_Sun])

### HIV

In [32]:
all_struct_HIV = pd.read_excel('data/HIV/nar-01265-r-2015-File010.xlsx', header=1)

def convert_to_CT(df, column, name, path):
    df_CT = df[['Position', 'Nucleotide ', column]].copy()

    df_CT['Position+1'] = df_CT['Position'] + 1
    df_CT['Position-1'] = df_CT['Position'] - 1
    df_CT[['Position', 'Nucleotide ', 'Position-1', 'Position+1', column, 'Position']].to_csv(path, sep='\t', index=False, header=[name, str(len(df))]+['']*4)


convert_to_CT(all_struct_HIV, column='Watts09', name='HIV', path='data/HIV/Watts/HIV.ct')
convert_to_CT(all_struct_HIV, column='PPfold, SHAPE', name='HIV', path='data/HIV/Sukosd/HIV.ct')

# Import shape file
UKN = -1000
data_shape = pd.read_csv('data/HIV/Sukosd/41586_2009_BFnature08237_MOESM312_ESM.txt', header=None, sep='\t')
data_shape.fillna(UKN, inplace=True)
full_shape_HIV = data_shape[2].values

In [33]:
# Import ct file from Watts
data = convert(
    'ct',
    'data/HIV/Watts/',
    name='HIV',
    path_out='',
    filter=False
)
df = pd.DataFrame(data).T
sequence_Watts = df['sequence']['HIV'].upper().replace('T','U')
paired_bases_Watts = np.array(df['structure']['HIV'])

assert ''.join(data_shape[1].tolist()).upper().replace('T','U') == sequence_Watts, 'Sequence mismatch'

Parsing ct files: 100%|██████████| 1/1 [00:00<00:00, 76.19it/s]

Drop 0 datapoints with None values (null sequence or reference)


In [34]:
# Import ct file from Sukosd
data = convert(
    'ct',
    'data/HIV/Sukosd/',
    name='HIV',
    path_out='',
    filter=False
)
df = pd.DataFrame(data).T
sequence_Sukosd = df['sequence']['HIV'].upper().replace('T','U')
paired_bases_Sukosd = np.array(df['structure']['HIV'])

assert ''.join(data_shape[1].tolist()).upper().replace('T','U') == sequence_Sukosd, 'Sequence mismatch'

Parsing ct files: 100%|██████████| 1/1 [00:00<00:00, 100.11it/s]

Drop 0 datapoints with None values (null sequence or reference)


In [39]:
compare_frags([sequence_Sukosd, sequence_Watts], [full_shape_HIV, full_shape_HIV], [paired_bases_Sukosd, paired_bases_Watts],
              name='HIV', min_length=40, min_unpaired_length=3, min_auroc=0.8, min_f1=0.8)

,sequence,dms
HIV_0,GGUCUCUCUGGUUAGACCAGAUCUGAGCCUGGGAGCUCUCUGGCUA...,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ..."
HIV_1,GGGAACCCACUGCUUAAGCCUCAAUAAAGCUUGCCUUGAGUGCUCA...,"[0.61, 0.05, 0.06, 0.04, 0.01, 0.03, 0.42, 0.0..."


## Convert to rouskinHF

In [15]:
import envbash
envbash.load.load_envbash('../.env')
from rouskinhf import convert, upload_dataset, get_dataset

## Pre-processing

In [6]:
import json

path = '/Users/yvesmartin/data/viral_fragments/viral_fragments.json'
raw = json.load(open(path))
data = {}

for k, v in raw.items():
    prefix = k
    for idx, (k2, v2) in enumerate(v.items()):
        data[f'{prefix}_{idx}'] = {'sequence': k2, 'dotbracket': v2}

json.dump(data, open('data/formatted_viral_fragments.json', 'w'), indent=2)
print(str(data)[:1000])

{'SARS_0': {'sequence': 'GACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGT', 'dotbracket': '.....((((...(((.((((((((.....((((((.(((((......)))))..)))))).........(((((((.((......)))))))))(((....)))))))))))))).))))'}, 'SARS_1': {'sequence': 'GAACGGTAATAAAGGAGCTGGTGGCCATAGTTACGGCGCCGATCTAAAGTCATTTGACTTAGGCGACGAGCTTGGCACTGATCCTTATGAAGATTTTCAAGAAAACTGGAACACTAAACA', 'dotbracket': '....((((....(((((..(((((.(((.((((.((.((((......(((((....))))).)))).)))))))))))))).))))).((((....)))).....))))...........'}, 'SARS_2': {'sequence': 'GCCCTGATGGCTACCCTCTTGAGTGCATTAAAGACCTTCTAGCACGTGCTGGTAAAGCTTCATGCACTTTGTCCGAACAACTGGACTTTATTGACACTAAGAGGGGTGTATACTGCTGCC', 'dotbracket': '((((.....)))).(((((((((((((((..(((.....(((((....))))).....))).)))))))).(((((......)))))............)))))))..............'}, 'SARS_3': {'sequence': 'CTGGACTTTATTGACACTAAGAGGGGTGTATACTGCTGCCGTGAACATGAGCATGAAATTGCTTGGTACACGGAACGTTCTGAAAAGAGCTATGAATTGCAGACACCTTTTGAAATT

## RouskinHF filtering

In [9]:
from rouskinhf import convert

data = convert(
    'json',
    'data/formatted_viral_fragments.json',
    name='viral_fragments',
    path_out='data',
    filter=True
)

Parsing json file: 100%|██████████| 53/53 [00:00<00:00, 36448.29it/s]

Over a total of 53 datapoints, there are:
### OUTPUT
- 53 valid datapoints
### MODIFIED
- 0 multiple sequences with the same reference (renamed reference)
- 0 duplicate sequences with different structure / dms / shape
### FILTERED OUT
- 0 invalid datapoints (ex: sequence with non-regular characters)
- 0 datapoints with bad structures
- 0 duplicate sequences with the same structure / dms / shape


## To HuggingFace

In [16]:
from rouskinhf import upload_dataset

upload_dataset(
    'data/viral_fragments/data.json',
    commit_message='init commit',
)